In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [2]:
# import dataset

df = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [4]:
# Definition
# Weighting Ratings : cosine similarity

from typing import List, Set, Dict, Tuple
import sklearn
from numpy.linalg import norm

def cosine_similarity(df:pd.DataFrame, userid_1:int, userid_2:int) -> List[int]:
    _df_1 = df[df['userId'] == userid_1]
    _df_2 = df[df['userId'] == userid_2]
    
    _ids_1 = [_id for _id in _df_1['movieId'][:10]]
    _ids_2 = [_id for _id in _df_2['movieId'][:10]]
    
    _ratings_1 = [_rate for _rate in _df_1['rating'][:10]]
    _ratings_2 = [_rate for _rate in _df_2['rating'][:10]]

    res_1 = pd.DataFrame({'movie_id' : _ids_1, 'rating' : _ratings_1}).to_numpy()
    res_2 = pd.DataFrame({'movie_id' : _ids_2, 'rating' : _ratings_2}).to_numpy()
    
    cosine = np.sum(res_1 * res_2, axis = 1) / (norm(res_1, axis = 1)*norm(res_2, axis = 1))
    print(cosine)

In [5]:
cosine_similarity(df, 131026, 95267)

[0.2838526  0.5636624  0.98916215 0.98862624 0.98726907 0.99574304
 0.98864669 0.99648569 0.99885099 0.99982687]
